In [24]:
!pip install langchain langchain-community chromadb sentence-transformers transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 6.2 MB/s eta 0:00:00


In [26]:
import os
from getpass import getpass

# Set your Hugging Face API key
if "HUGGINGFACE_API_KEY" not in os.environ:
    os.environ["HUGGINGFACE_API_KEY"] = getpass("Hugging Face API key (input hidden): ")

# Optional: allow user to set model name
# You can change this to another Hugging Face model, e.g. "mistralai/Mistral-7B-Instruct-v0.2"
os.environ["HF_MODEL"] = os.getenv("HF_MODEL", "facebook/bart-large-cnn")

# Twilio (for WhatsApp/SMS). If you don't plan to use Twilio, skip these.
if "TWILIO_ACCOUNT_SID" not in os.environ:
    os.environ["TWILIO_ACCOUNT_SID"] = getpass("Twilio Account SID (or leave blank): ")
if "TWILIO_AUTH_TOKEN" not in os.environ:
    os.environ["TWILIO_AUTH_TOKEN"] = getpass("Twilio Auth Token (or leave blank): ")
if "TWILIO_WHATSAPP_FROM" not in os.environ:
    # Twilio Sandbox 'whatsapp:+14155238886' or your registered number
    os.environ["TWILIO_WHATSAPP_FROM"] = input("Twilio WhatsApp 'From' (e.g. whatsapp:+14155238886) or leave blank: ")


Hugging Face API key (input hidden): ··········


In [27]:
import os, io, time
from typing import List
from pathlib import Path
from google.colab import files

# LangChain imports
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.schema import Document

# small helper to show progress
def print_sep(s=""):
    print("\n" + "="*10 + " " + s + " " + "="*10 + "\n")

In [28]:
!pip install -U langchain-community

In [29]:
import os, io, time
from typing import List
from pathlib import Path
from google.colab import files

# LangChain imports
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings # Updated import
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.schema import Document

# small helper to show progress
def print_sep(s=""):
    print("\n" + "="*10 + " " + s + " " + "="*10 + "\n")

In [30]:
print("Upload files (PDF/CSV/TXT/XLSX/MD/HTML). Use Ctrl/Cmd to select multiple files.")
uploaded = files.upload()  # browser upload

uploaded_paths = []
for fn in uploaded.keys():
    path = "/content/" + fn
    uploaded_paths.append(path)
    # write file bytes
    with open(path, "wb") as f:
        f.write(uploaded[fn])
    print("Saved", path)

print("Files ready:", uploaded_paths)

Upload files (PDF/CSV/TXT/XLSX/MD/HTML). Use Ctrl/Cmd to select multiple files.


Saving train.csv to train (1).csv
Saved /content/train (1).csv
Files ready: ['/content/train (1).csv']


In [31]:
# Colab cell 5
import pandas as pd
from pypdf import PdfReader
from langchain.schema import Document

def load_file_to_documents(path: str) -> List[Document]:
    path = Path(path)
    ext = path.suffix.lower()
    docs = []
    if ext == ".pdf":
        reader = PdfReader(str(path))
        text = ""
        for p in reader.pages:
            page_text = p.extract_text()
            if page_text:
                text += page_text + "\n\n"
        docs.append(Document(page_content=text, metadata={"source": str(path)}))

    elif ext in [".txt", ".md"]:
        text = path.read_text(encoding="utf-8", errors="ignore")
        docs.append(Document(page_content=text, metadata={"source": str(path)}))

    elif ext == ".csv":
        df = pd.read_csv(path)
        # convert dataframe rows to one txt blob so retrieval works
        text = df.to_csv(index=False)
        docs.append(Document(page_content=text, metadata={"source": str(path)}))

    elif ext in [".xls", ".xlsx"]:
        df = pd.read_excel(path)
        text = df.to_csv(index=False)
        docs.append(Document(page_content=text, metadata={"source": str(path)}))

    elif ext in [".html", ".htm"]:
        text = path.read_text(encoding="utf-8", errors="ignore")
        # optionally strip tags. For small files ok to keep raw html
        docs.append(Document(page_content=text, metadata={"source": str(path)}))
    else:
        # fallback: read bytes
        try:
            text = path.read_text(encoding="utf-8", errors="ignore")
            docs.append(Document(page_content=text, metadata={"source": str(path)}))
        except Exception as e:
            print("Unsupported file type for", path, ":", e)
    return docs

# load all uploaded files
all_docs = []
for p in uploaded_paths:
    all_docs += load_file_to_documents(p)

print("Loaded documents:", [d.metadata for d in all_docs])


Loaded documents: [{'source': '/content/train (1).csv'}]


In [32]:
# Colab cell 6
# If you want to pull web pages directly (example):
import requests
from bs4 import BeautifulSoup

def url_to_document(url: str) -> Document:
    r = requests.get(url, timeout=15)
    r.raise_for_status()
    html = r.text
    soup = BeautifulSoup(html, "html.parser")
    # get readable text
    text = "\n\n".join([p.get_text().strip() for p in soup.select("p") if p.get_text().strip()])
    return Document(page_content=text, metadata={"source": url})

# Example: add WHO FAQ page
# doc = url_to_document("https://www.who.int/news-room/fact-sheets")
# all_docs.append(doc)


In [33]:
import requests
from bs4 import BeautifulSoup
from langchain.schema import Document

# Function to convert a single URL into a Document
def url_to_document(url: str) -> Document:
    r = requests.get(url, timeout=15)
    r.raise_for_status()
    html = r.text
    soup = BeautifulSoup(html, "html.parser")
    # Extract readable text (from <p> tags)
    text = "\n\n".join([p.get_text().strip() for p in soup.select("p") if p.get_text().strip()])
    return Document(page_content=text, metadata={"source": url})

# --- Add your URLs here ---
urls = [
    "https://www.who.int/news-room/fact-sheets",
    "https://www.cdc.gov/coronavirus/2019-ncov/faq.html",
    "https://www.nih.gov/health-information",
    "https://www.aap.org/en/patient-care/infection-prevention-and-control/project-firstline/covid-19-infection-prevention-and-control-frequently-asked-questions",
    "https://indianexpress.com/article/lifestyle/health/anopheles-mosquito-malaria-causes-symptoms-treatment-5871760/",
    "https://indianexpress.com/article/lifestyle/health/anopheles-mosquito-malaria-causes-symptoms-treatment-5871760/",
    "https://ckphu.com/health-topics/diseases-infections/disease-and-infections-faqs/"
]

# Convert all URLs into Documents
all_docs = []
for url in urls:
    try:
        doc = url_to_document(url)
        all_docs.append(doc)
        print(f"✅ Loaded: {url}")
    except Exception as e:
        print(f"❌ Error loading {url}: {e}")

print(f"\nTotal documents loaded: {len(all_docs)}")

✅ Loaded: https://www.who.int/news-room/fact-sheets
❌ Error loading https://www.cdc.gov/coronavirus/2019-ncov/faq.html: 404 Client Error: Not Found for url: https://www.cdc.gov/coronavirus/2019-ncov/faq.html
✅ Loaded: https://www.nih.gov/health-information
❌ Error loading https://www.aap.org/en/patient-care/infection-prevention-and-control/project-firstline/covid-19-infection-prevention-and-control-frequently-asked-questions: HTTPSConnectionPool(host='www.aap.org', port=443): Max retries exceeded with url: /en/patient-care/infection-prevention-and-control/project-firstline/covid-19-infection-prevention-and-control-frequently-asked-questions (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f28ec353080>, 'Connection to www.aap.org timed out. (connect timeout=15)'))
✅ Loaded: https://indianexpress.com/article/lifestyle/health/anopheles-mosquito-malaria-causes-symptoms-treatment-5871760/
✅ Loaded: https://indianexpress.com/article/lifestyle/health/anopheles-m

In [34]:
# Colab cell 7
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
print("Number of source docs:", len(all_docs))
docs_chunks = text_splitter.split_documents(all_docs)
print("Number of chunks:", len(docs_chunks))
# look at a single chunk (optional)
print("\n--- Sample chunk ---\n", docs_chunks[0].page_content[:800])


Number of source docs: 5
Number of chunks: 26

--- Sample chunk ---
 Abortion

Abuse of older people

Adolescent and young adult health

Adolescent pregnancy

Ageing and health

Alcohol

Ambient (outdoor) air pollution

Anaemia

Animal bites

Antimicrobial resistance

Anxiety disorders

Arsenic

Asbestos

Assistive technology

Asthma

Autism

Bacterial vaginosis

Biodiversity

Bipolar disorder

Blindness and vision impairment

Blood safety and availability

Botulism

Breast cancer

Brucellosis

Burns

Buruli ulcer (Mycobacterium ulcerans infection)

Campylobacter

Cancer

Candidiasis (yeast infection)

Cardiovascular diseases (CVDs)

Cervical cancer

Chagas disease (also known as American trypanosomiasis)

Chikungunya

Child maltreatment

Child mortality (under 5 years)

Childhood cancer

Chlamydia

Cholera

Chromoblastomycosis

Chronic obstructive pulmonar


In [38]:
# Colab cell 8 (Hugging Face version)
from langchain_community.embeddings import HuggingFaceEmbeddings # Corrected import
from langchain_chroma import Chroma

persist_dir = "/content/chroma_db"

print("Creating embeddings (Hugging Face)...")
# You can change model name if needed (e.g. "intfloat/e5-small-v2" for faster results)
emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

print("Indexing into Chroma DB (this stores on disk in persist_dir)...")
vectordb = Chroma.from_documents(documents=docs_chunks, embedding=emb, persist_directory=persist_dir)
vectordb.persist()
print("Indexed & persisted at:", persist_dir)

ModuleNotFoundError: No module named 'langchain_chroma'

In [39]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

persist_dir = "/content/chroma_db"

print("Creating embeddings (HuggingFace, local)...")
emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

print("Indexing into Chroma DB (this stores on disk in persist_dir)...")
vectordb = Chroma.from_documents(documents=docs_chunks, embedding=emb, persist_directory=persist_dir)
vectordb.persist()
print("✅ Indexed & persisted at:", persist_dir)


Creating embeddings (HuggingFace, local)...
Indexing into Chroma DB (this stores on disk in persist_dir)...
✅ Indexed & persisted at: /content/chroma_db


In [41]:
# Colab cell 9
'''
llm = ChatOpenAI(temperature=0, model=os.getenv("OPENAI_MODEL","gpt-4o-mini"))

retriever = vectordb.as_retriever(search_kwargs={"k":4})
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
print("QA chain ready. Try qa_chain.run('your question')")
'''

QA chain ready. Try qa_chain.run('your question')


In [53]:
!pip install huggingface_hub==0.20.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.34.4
    Uninstalling huggingface-hub-0.34.4:
      Successfully uninstalled huggingface-hub-0.34.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 0.3.1 requires huggingface-hub>=0.33.4, but you have huggingface-hub 0.20.3 which is incompatible.
transformers 4.56.1 requires huggingface-hub<1.0,>=0.34.0, but you have huggingface-hub 0.20.3 which is incompatible.
gradio 5.44.1 requires huggingface-hub<1.0,>=0.33.5, but you have huggingface-hub 0.20.3 which is incompatible.
accelerate 1.10.1 requires huggingface_hub>=0.21.0, but you have huggingface-hub 0.20.3 which is incompatible.
diffusers 0.35.1 requires huggingface-hub>=0.34.0, but you have huggingface-hub 0.20.3 which

In [58]:
# Colab cell 10 (fixed Hugging Face wrapper with Pydantic fields)

from huggingface_hub import InferenceClient
from langchain.llms.base import LLM
from langchain.chains import RetrievalQA
from typing import Optional, List
from pydantic import Field

# Proper custom wrapper for Hugging Face text-generation models
class HuggingFaceLLM(LLM):
    model: str = Field(...)
    token: str = Field(...)
    max_new_tokens: int = Field(default=512)
    temperature: float = Field(default=0.3)

    @property
    def _llm_type(self) -> str:
        return "huggingface_llm"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        client = InferenceClient(model=self.model, token=self.token)
        response = client.text_generation(
            prompt,
            max_new_tokens=self.max_new_tokens,
            temperature=self.temperature,
            stop_sequences=stop
        )
        return response

# Load LLM
llm = HuggingFaceLLM(
    model="mistralai/Mistral-7B-Instruct-v0.2",  # try "google/flan-t5-base" if this is too heavy
    token=os.environ["HUGGINGFACE_API_KEY"],
    max_new_tokens=512,
    temperature=0.3
)

# Build RetrievalQA with Chroma retriever
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(),
    chain_type="stuff"
)

# Run queries
queries = [
    "What are the common symptoms of dengue?",
    "How can I prevent malaria at home?",
    "What is a safe first aid step for minor burns?"
]

for q in queries:
    print("="*50)
    print("Q:", q)
    try:
        ans = qa_chain.run(q)
    except Exception as e:
        ans = f"Error running chain: {e}"
    print("A:", ans)


Q: What are the common symptoms of dengue?
A: Error running chain: Model mistralai/Mistral-7B-Instruct-v0.2 is not supported for task text-generation and provider featherless-ai. Supported task: conversational.
Q: How can I prevent malaria at home?
A: Error running chain: Model mistralai/Mistral-7B-Instruct-v0.2 is not supported for task text-generation and provider featherless-ai. Supported task: conversational.
Q: What is a safe first aid step for minor burns?
A: Error running chain: Model mistralai/Mistral-7B-Instruct-v0.2 is not supported for task text-generation and provider featherless-ai. Supported task: conversational.


In [45]:
!pip install -U langchain-huggingface

In [47]:
# Colab cell 11
from flask import Flask, request
from twilio.twiml.messaging_response import MessagingResponse
from pyngrok import ngrok
import threading

app = Flask(__name__)

DISCLAIMER = ("\n\n[Disclaimer: This bot provides general health information only — "
              "not medical advice. For emergencies or personal medical concerns, "
              "seek a healthcare professional.]")

@app.route("/twilio-webhook", methods=["POST"])
def twilio_webhook():
    incoming_msg = request.values.get('Body', '').strip()
    from_number = request.values.get('From', '')
    print(f"Incoming from {from_number}: {incoming_msg}")

    # generate answer from RAG chain
    try:
        answer = qa_chain.run(incoming_msg)
    except Exception as e:
        answer = "Sorry, I couldn't fetch an answer right now. " + str(e)

    resp = MessagingResponse()
    resp.message(answer + DISLAIMER)
    return str(resp)

# Run flask in background thread
def run_app():
    app.run(host="0.0.0.0", port=5000)

t = threading.Thread(target=run_app, daemon=True)
t.start()

# start ngrok tunnel so Twilio can call this Colab instance
public_url = ngrok.connect(5000)
print("Public URL (use this for Twilio webhook):", public_url.public_url)
print("Webhook endpoint:", public_url.public_url + "/twilio-webhook")


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


ERROR:pyngrok.process.ngrok:t=2025-09-10T15:13:47+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-09-10T15:13:47+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-09-10T15:13:47+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [ ]:
# Colab cell 12
# To reload a previously persisted DB:
emb = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_dir, embedding_function=emb)
retriever = vectordb.as_retriever(search_kwargs={"k":4})
qa_chain = RetrievalQA.from_chain_type(llm=ChatOpenAI(temperature=0, model=os.getenv("OPENAI_MODEL")), retriever=retriever, chain_type="stuff")
print("Reloaded vectordb and QA chain.")
